In [10]:
from models import MLDataset
from utils.paths import Paths
from facades import EvaluationFacade
from datetime import datetime

lasso_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_LASSO_to_2021')
lgbm_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_LGBM_to_2021')
en_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_ensembled_to_2021')

#lasso_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_LASSO_learned_in_250408')
#lgbm_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_LGBM_learned_in_250408')
#en_dataset = MLDataset(f'{Paths.ML_DATASETS_FOLDER}/48sectors_Ensembled_learned_in_250408')

In [11]:
#en_dataset.evaluation_materials.pred_result_df['Pred'] = \
#    lasso_dataset.evaluation_materials.pred_result_df['Pred'] * 1.3 + \
#    lgbm_dataset.evaluation_materials.pred_result_df['Pred'] * 11.3
en_dataset.evaluation_materials.pred_result_df

,,Target,Pred
Date,Sector,,
2014-01-06,ITインフラ,0.000203,47.0
2014-01-07,ITインフラ,-0.008966,48.0
2014-01-08,ITインフラ,-0.005443,16.0
2014-01-09,ITインフラ,-0.002799,2.0
2014-01-10,ITインフラ,-0.000299,46.0
...,...,...,...
2025-04-02,食品,-0.005437,37.0
2025-04-03,食品,0.008438,31.0
2025-04-04,食品,0.018110,39.0


In [14]:
ml_dataset = lgbm_dataset
#ml_dataset =MLDataset(f'{Paths.ML_DATASETS_FOLDER}/LGBM_New48sectors_Ensembled_to2021')
ef = EvaluationFacade(ml_dataset, 
                 start_day = datetime(2022, 1, 1), 
                 end_day = datetime(2025, 12, 31),
                 bin_num=5)
ef.display()

c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


読み込みが完了しました。
全期間
母集団の推定（99.7%）
平均リターン: 0.0903% (worst: 0.0312%, best: 0.1494%)
標準偏差: 0.005613 (worst:0.00606, best:0.005222)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.160885 (worst:0.051408, best:0.286185)
最大ドローダウン: 7.8494% (worst:26.5226%, best:4.1056%)
最大ドローダウン日数: 348日 (worst:3405日, best:110日)
----------------------------------------------------------------------------------
直近1年
母集団の推定（99.7%）
平均リターン: 0.0624% (worst: -0.0394%, best: 0.1642%)
標準偏差: 0.005393 (worst:0.006204, best:0.004756)
-----------------------------------------
モデルのスペック
シャープレシオ: 0.115748 (worst:-0.063499, best:0.345319)
最大ドローダウン: 10.4831% (worst:-21.982%, best:3.0989%)
最大ドローダウン日数: 672日 (worst:2232日, best:75日)
----------------------------------------------------------------------------------
直近3ヶ月
母集団の推定（99.7%）
平均リターン: -0.0083% (worst: -0.2348%, best: 0.2183%)
標準偏差: 0.005821 (worst:0.007851, best:0.004577)
-----------------------------------------
モデルのスペック
シャープレシオ: -0.01418 (worst:-0.299094, best:

Output()

In [3]:
df = ef.visualizer.metrics_dfs['日次成績'][['LS']].rename(columns={'LS': 'Target'})
df.to_csv('result_現行.csv')